In [ ]:
import os
import urllib.request
import numpy as np
import torch
from models import *
from utils import *
import matplotlib.pyplot as plt

MODEL_URL  = "https://github.com/hasanjawad001/rheed-universal-translator/releases/download/v1.0/model_weights.pth"
MODEL_PATH = "outputs/model_weights.pth"
META_PATH  = "outputs/model_meta.npz"

os.makedirs("outputs", exist_ok=True)

if not os.path.isfile(MODEL_PATH):
    print("Model weights not found. Downloading...")
    urllib.request.urlretrieve(MODEL_URL, MODEL_PATH)
    print("Model download complete.")
else:
    print("Model weights found. Skipping download.")

# ---- load metadata ----
meta = np.load(META_PATH)
min_stoich = float(meta["min_stoich"])
max_stoich = float(meta["max_stoich"])
min_image  = float(meta["min_image"])
max_image  = float(meta["max_image"])
IMG_HW     = tuple(meta["IMG_HW"])

loaded_data = np.load('inputs/rheed_stoich_data.npz')
stoich = loaded_data['stoich'].astype(np.float32)
images = loaded_data['images'].astype(np.float32)

print(stoich.shape, images.shape)

# ---- load model ----
device = "cuda" if torch.cuda.is_available() else "cpu"
model = UniversalTranslator(IMG_HW).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# ===============================
# CASE 1: Image → (Image + Sto)
# ===============================

# use first sample
first_img = images[0]          # input: RHEED image
first_sto = float(stoich[0])   # ground-truth stoichiometry

# ---- inference using IMAGE as input ----
# Model predicts:
#   (1) reconstructed image (image → image)
#   (2) stoichiometry (image → sto)
img_pred, sto_pred = predict_from_input(
    model=model,
    input_data=first_img,
    input_type="img",
    IMG_HW=IMG_HW,
    min_stoich=min_stoich,
    max_stoich=max_stoich,
    min_image=min_image,
    max_image=max_image,
    device=device
)

print("Input: RHEED image → Output: reconstructed RHEED image and stoichiometry")
print()
print(
    f"[Image → (Image + Sto)] "
    f"True sto: {first_sto:.4f} | Pred sto: {sto_pred:.4f}"
)

# ---- visualize ----
fig, axes = plt.subplots(1, 2, figsize=(8,4))

axes[0].imshow(first_img, cmap="viridis")
axes[0].set_title(f"True Image (Input)")
axes[0].axis("off")

axes[1].imshow(img_pred, cmap="viridis")
axes[1].set_title(f"Reconstructed Image (I→I)")
axes[1].axis("off")

plt.tight_layout()
plt.show()


# ==================================
# CASE 2: Sto → (Image + Sto)
# ==================================

# use last sample
last_img = images[-1]          # ground-truth RHEED image (for reference)
last_sto = float(stoich[-1])   # input: stoichiometry

# ---- inference using STOICHIOMETRY as input ----
# Model predicts:
#   (1) RHEED image (sto → image)
#   (2) reconstructed stoichiometry (sto → sto)
img_pred, sto_pred = predict_from_input(
    model=model,
    input_data=last_sto,
    input_type="sto",
    IMG_HW=IMG_HW,
    min_stoich=min_stoich,
    max_stoich=max_stoich,
    min_image=min_image,
    max_image=max_image,
    device=device
)

print("Input: stoichiometry value → Output: RHEED image and reconstructed stoichiometry")
print()
print(
    f"[Sto → (Image + Sto)] "
    f"True sto (Input): {last_sto:.4f} | Recon sto: {sto_pred:.4f}"
)

# ---- visualize ----
fig, axes = plt.subplots(1, 2, figsize=(8,4))

axes[0].imshow(last_img, cmap="viridis")
axes[0].set_title(f"True Image")
axes[0].axis("off")

axes[1].imshow(img_pred, cmap="viridis")
axes[1].set_title(f"Predicted Image (Sto→I)")
axes[1].axis("off")

plt.tight_layout()
plt.show()
